# Essai avec spacy


In [1]:
import numpy as np
import pandas as pd
import json
import glob



In [2]:
import spacy
from spacy.matcher import PhraseMatcher

In [3]:
root_path = '/Users/juliettemontanteme/Desktop/data_covid'
#astuce : pour trouver facilement le chemin, ouvrir le dossier correspondant 
#et en appuyant sur "alt" aller sur édition et copier le chemin
metadata_path = f'{root_path}/metadata.csv'
meta_df = pd.read_csv(metadata_path)

/Users/juliettemontanteme/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


On code tout d'abord les fonctions de base : ouverture article, renvoi du texte, création de la base des articles ayant un texte. 

In [6]:
def has_full_text(x):
    if x["pdf_json_files"]==True and x["pmc_json_files"]==True:
        return False
    return True

In [7]:
meta_bool=meta_df[["pdf_json_files","pmc_json_files"]].isnull()
meta_bool["has_full"]=meta_bool.apply(lambda x:has_full_text(x),axis=1)
meta_df_full_text=meta_df.loc[meta_bool["has_full"]]


In [131]:
n = len(meta_df_full_text)
n

68376

On va maintenant passer à la recherche de mots-clés.

Avant de commencer, on effectue un préprocessing rapide des données (on peut ajouter du lemmatizing et enlever les stops words si besoin)

In [26]:
#Chargement de la base de données anglaise
nlp = spacy.blank('en')

In [102]:
#Classe permettant de faire un preprocessing d'un texte

class Preprocess:
    def __init__(self, path):
        self.path = path
        self.article = ""
        self.text = ""
        self.text1 = ""
        self.text2 = ""
    def open_article(self):
        file=open(str(root_path)+"/"+str(path),'r')
        article=json.load(file)
        file.close()
        self.article = article
    
    def make_text(self):
        text_list = []
        for entry in self.article['body_text']:
            text_list.append(entry['text'])
            text_list.append("\n")
        text_full=''.join(text_list)
        self.text = text_full
    
    def preprocess(self): 
        self.text1 = nlp(self.text)
        text2 = ""
        for token in self.text1: 
            if token.is_stop == False:
                text2 = text2 + " " + token.lemma_
        self.text2 = nlp(text2)

    def repr_1(self):
        return(self.text1)
    
    def repr_2(self):
        return(self.text2)

#texts = []
#i = 0
#while i < n:
    #path = meta_df_full_text.iloc[i]['pmc_json_files']
    #if str(path)[0] != "d":
        #i = i + 1
    #else : 
        #article = open_article(path)
        #text = make_text(article)
        #texts.append(nlp(text))
        #i = i + 1




In [103]:
path = meta_df_full_text.iloc[1]['pmc_json_files']


In [104]:
proc = Preprocess(path)
proc.open_article()
proc.make_text()
proc.preprocess()
doc = proc.repr_2()

In [105]:
doc

 discovery biological messenger molecule 10 years ago , gaseous molecule nitric oxide ( NO• ) recognized involvement diverse biological processes , including vasodilation , bronchodilation , neurotransmission , tumor surveillance , antimicrobial defense regulation inflammatory - immune processes [ 1,2,3 ] . respiratory tract , NO• generated enzymically distinct isoforms NO• synthase ( NOS-1 , NOS-2 NOS-3 ) present different extents numerous cell types , including airway alveolar epithelial cells , neuronal cells , macrophages , neutrophils , mast cells , endothelial smooth - muscle cells . contrast NOS isoforms ( NOS-1 NOS-3 ) , expressed constitutively activated mediator - induced stress - induced cell activation , NOS-2 activity primarily regulated transcriptionally commonly induced bacterial products pro - inflammatory cytokines . , inflammatory diseases respiratory tract , asthma , acute respiratory distress syndrome ( ARDS ) bronchiectasis , commonly characterized increased expres

In [106]:
matcher = PhraseMatcher(nlp.vocab, attr='LOWER')

In [107]:
hypertension = ['hypertension', 'high blood pressure', 'hypertensive', 'high pressure', 'raised blood pressure', 'htn', 'hbp', 'ht', ]
diabetes = ['diabetes', 'high blood sugar', 'insulin resistance', 'diabetic', 'diabetics', 'dm']
gender = ['male gender', 'male', 'gender', 'sex', 'masculine', 'female']
heart_disease = ['cardiopathy', 'heart disease', 'heart', 'chd', 'arrhythmia', 'tachycardia', 'bradycardia', 'fibrillation', 'cardiomyopathy', 'infarction']
COPD_respiratory_system = ['copd', 'emphysema', 'bronchitis', 'asthma', 'bronchiectasis', 'respiratory', 'trachea', 'lung', 'lungs', 'pulmonary']
smoking_status = ['smoking', 'smoke', 'smoker', 'tobacco']
age = ['age', 'old', 'young', 'senior', 'child', 'children']
cerebrovascular_disease = ['cerebrovascular', 'embolism', 'ischemic', 'stroke', 'aneurysm', 'tia', 'subarachnoid']
cancer = ['cancer', 'leukemia', 'cancerology']
kidney_disease = ['kidney', 'edema', 'anemia', 'gfr', 'dialysis']
drinking = ['drinking', 'alcohol', 'alcoholic']
overweight = ['overweight', 'obesity', 'obese', 'bmi']
liver_disease = ['liver', 'fascioliasis', 'cirrhosis', 'hepatitis', 'psc']

In [108]:
hta_tokens_list = [nlp(factor) for factor in hypertension]
diabetes_tokens_list = [nlp(factor) for factor in diabetes]
gender_tokens_list = [nlp(factor) for factor in gender]
heart_tokens_list = [nlp(factor) for factor in heart_disease]
respi_tokens_list = [nlp(factor) for factor in COPD_respiratory_system]
smoking_tokens_list = [nlp(factor) for factor in smoking_status]
age_tokens_list = [nlp(factor) for factor in age]
cerebrovascular_tokens_list = [nlp(factor) for factor in cerebrovascular_disease]
cancer_tokens_list = [nlp(factor) for factor in cancer]
kidney_tokens_list = [nlp(factor) for factor in kidney_disease]
drinking_tokens_list = [nlp(factor) for factor in drinking]
overweight_tokens_list = [nlp(factor) for factor in overweight]
liver_tokens_list = [nlp(factor) for factor in liver_disease]



In [84]:
matcher.add("HTA",hta_tokens_list)
matcher.add("DIABETES",diabetes_tokens_list)
matcher.add("GENDER",gender_tokens_list )
matcher.add("HEART",heart_tokens_list )
matcher.add("RESPIRATORY",respi_tokens_list )
matcher.add("SMOKING",smoking_tokens_list )
matcher.add("AGE",age_tokens_list )
matcher.add("CEREBROVASCULAR",cerebrovascular_tokens_list )
matcher.add("CANCER",cancer_tokens_list )
matcher.add("KIDNEY",kidney_tokens_list )
matcher.add("DRINKING",drinking_tokens_list )
matcher.add("OVERWEIGHT",overweight_tokens_list )
matcher.add("LIVER",liver_tokens_list )


In [109]:
matcher.add("RESPIRATORY",respi_tokens_list )

In [110]:
matches = matcher(doc)

In [111]:
import math

In [130]:
class Diffusivity:
    def __init__(self, matches, doc):
        self.matches = matches
        self.doc = doc
        self.positions = []
        self.occurence = len(self.matches)
        
    def position(self):
        for match in self.matches :
            pos = (match[1] + match[2])/2
            self.positions.append(int(pos))

        
    def score(self):
        mean_pos = np.mean(self.positions)
        ecart_pos = math.sqrt(np.var(self.positions))
        score = ecart_pos / len(self.doc)
        return score
        

            

In [132]:
dif = Diffusivity(matches, doc)
dif.position()
dif.score()

0.3420907247640987

In [86]:
found_factors = [doc[match[1]:match[2]] for match in matches]
print(found_factors)

[respiratory, respiratory, respiratory, respiratory, asthma, asthma, respiratory, respiratory, bronchiectasis, bronchiectasis, respiratory, respiratory, lung, lung, lung, lung, respiratory, respiratory, lung, lung, lung, lung, lung, lung, asthma, asthma, asthma, asthma, asthma, asthma, lung, lung, lung, lung, respiratory, respiratory, respiratory, respiratory, respiratory, respiratory, lung, lung, lung, lung, lung, lung, respiratory, respiratory, respiratory, respiratory, asthma, asthma, pulmonary, pulmonary, lung, lung, respiratory, respiratory, lung, lung, asthma, asthma, lung, lung, lung, lung, respiratory, respiratory, lung, lung, lung, lung, asthma, asthma, asthma, asthma, lung, lung, lung, lung, respiratory, respiratory, respiratory, respiratory, lung, lung, lung, lung]


In [133]:
class Frequency: 
    def __init__(self, matches, doc):
        self.matches = matches
        self.doc = doc
        self.occurence = len(self.matches)
    
    def score(self):
        return len(self.matches)/len(self.doc)


In [135]:
fr = Frequency(matches, doc)
fr.score()

0.02030456852791878

In [111]:
paths =list(meta_df_full_text["pdf_json_files"])



'document_parses/pdf_json/6ff8826ca04c185f3a88ab4b0d5a912aea4ab1a6.json'

['document_parses/pdf_json/d1aafb70c066a2068b02786f8929fd9c900897fb.json',
 'document_parses/pdf_json/6b0567729c2143a66d737eb0a2f63f2dce2e5a7d.json',
 'document_parses/pdf_json/06ced00a5fc04215949aa72528f2eeaae1d58927.json',
 'document_parses/pdf_json/348055649b6b8cf2b9a376498df9bf41f7123605.json',
 'document_parses/pdf_json/5f48792a5fa08bed9f56016f4981ae2ca6031b32.json',
 'document_parses/pdf_json/b2897e1277f56641193a6db73825f707eed3e4c9.json',
 'document_parses/pdf_json/3bb07ea10432f7738413dff9816809cc90f03f99.json',
 'document_parses/pdf_json/5806726a24dc91de3954001effbdffd7a82d54e2.json',
 'document_parses/pdf_json/faaf1022ccfe93b032c5608097a53543ba24aedb.json',
 'document_parses/pdf_json/5b44feca5d6ffaaeb66501fa84cc6dd44d06660a.json']

In [112]:

for path in paths: 
    l = str(path).split(';')
    for new_path in l:
        paths.append(new_path)



KeyboardInterrupt: 

In [96]:
meta_df_full_text["pmc_json_files"][76]


'document_parses/pmc_json/PMC1636417.xml.json'

In [ ]:
list_of_texts =[]
for i in range(n):
    try:
        path = meta_df_full_text["pmc_json_files"][i]
    except KeyError:
        print(f'La clé {i} n existe pas')
    else : 
        try: 
            article=open_article(str(path))
        except FileNotFoundError:
            print(f'{i} and {path}')
        else: 
            text = make_text(article)
            list_of_texts.append(text)



In [86]:

str(meta_df_full_text["pdf_json_files"][78])






'document_parses/pdf_json/52566dccb4bd8044edc87b1a0aa268320a6ea3d4.json; document_parses/pdf_json/8b39433dd865c0f71c7b2f333e1f506b73d722f1.json'

In [76]:
meta_df_full_text["pdf_json_files"][71]

'document_parses/pdf_json/c1ad13d83e926979dbf2bbe52e4944082f28dfea.json'